In [1]:
from DataLoader import DataLoader
from Anonymisation import Anonymisation
from Consistenter import Consistenter
from GUM import GraduallyUpdateMethod
from PostProcessor import RecordPostprocessor

In [2]:
dl = DataLoader('Polish/polish.csv', 'Polish/data.yaml', 'Polish/column_info.json', 'Polish/loading_data.json')
dl.data_loader()
dl.all_indifs(dl.private_data)
pass

/Users/finalsukjin/Desktop/Sukrit Jindal PrivSyn Code - 22125037, O10/DataLoader.py:99: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  self.private_data.fillna(self.fillna, inplace=True, downcast = self.datatypes)


,smoke,sex,age,edu,weight,height,bmi,sport,marital,region,wkabint,income,ls
0,1,1,5,0,6,6,2,0,0,0,0,1,0
1,1,0,1,0,5,8,2,0,1,1,0,1,1
2,1,1,1,0,2,5,1,0,1,2,0,0,0
3,1,1,7,1,5,5,2,1,2,1,0,1,2
4,0,1,4,0,2,5,1,1,0,3,0,1,1


In [3]:
anon = Anonymisation(1,3e-11)
anon.anonymiser(dl)
pass

epsilon: 1, delta: 3e-11, rho: 0.010110284535384867
one-way sigma: 80.18165003549409
indif sigma: 2.3399999999999998e-09
multi-way sigma: 45.84561237311878


In [4]:
c = Consistenter(anon, dl.all_attrs)
c.make_consistent(iterations = 0)
pass

In [5]:
gum = GraduallyUpdateMethod(dl, c)
gum.initialiser(view_iterations = 100)
syn_data = gum.synthesize(iterations = 100, num_records = int(c.num_synthesize_records))
pass

Iteration 1 of 100 completed to consist the marginal views
Iteration 2 of 100 completed to consist the marginal views
Iteration 3 of 100 completed to consist the marginal views
Iteration 4 of 100 completed to consist the marginal views
Iteration 5 of 100 completed to consist the marginal views
Iteration 6 of 100 completed to consist the marginal views
Iteration 7 of 100 completed to consist the marginal views
Iteration 8 of 100 completed to consist the marginal views
Iteration 9 of 100 completed to consist the marginal views
Iteration 10 of 100 completed to consist the marginal views
Iteration 11 of 100 completed to consist the marginal views
Iteration 12 of 100 completed to consist the marginal views
Iteration 13 of 100 completed to consist the marginal views
Iteration 14 of 100 completed to consist the marginal views
Iteration 15 of 100 completed to consist the marginal views
Iteration 16 of 100 completed to consist the marginal views
Iteration 17 of 100 completed to consist the marg

In [6]:
processor_priv = RecordPostprocessor(dl.private_data, dl.configpath, dl.datainfopath, dl.decode_mapping)
processor_pub = RecordPostprocessor(syn_data, dl.configpath, dl.datainfopath, dl.decode_mapping)
priv_data = processor_priv.post_process()
pub_data = processor_pub.post_process()

## Marginal Difference Score

In [7]:
t = 0
for i in range(len(dl.all_attrs)):
    for j in range(i+1, len(dl.all_attrs)):
        attr1 = dl.all_attrs[i]
        attr2 = dl.all_attrs[j]
        priv_marg = dl.two_way_marginal(attr1, attr2, dl.private_data)/len(dl.private_data)
        pub_marg = dl.two_way_marginal(attr1, attr2, syn_data)/len(syn_data)
        t += (priv_marg - pub_marg).abs().sum().sum()
t/(len(dl.all_attrs)*(len(dl.all_attrs)-1))

0.1464433984998855

## Classification Score

In [8]:
from sklearn import svm
import pandas as pd

In [9]:
priv_x = priv_data.copy(deep = True).drop(columns = ['income'])
priv_x_ohe = pd.get_dummies(priv_x)
priv_y = priv_data['income']

In [10]:
pub_x = pub_data.copy(deep = True).drop(columns = ['income'])
pub_x_ohe = pd.get_dummies(pub_x)
for col in priv_x_ohe.columns:
    if col not in pub_x_ohe.columns:
        pub_x_ohe[col]=0
cols = priv_x_ohe.columns
pub_x_ohe = pub_x_ohe[cols]
pub_y = pub_data['income']

In [11]:
priv_classifier = svm.SVC()
priv_classifier.fit(priv_x_ohe, priv_y)
priv_score = priv_classifier.score(priv_x_ohe, priv_y)
priv_misscore = 1-priv_score
priv_misscore

0.36547497446373856

In [12]:
pub_classifier = svm.SVC()
pub_classifier.fit(pub_x_ohe, pub_y)
pub_score = pub_classifier.score(priv_x_ohe, priv_y)
pub_misscore = 1-pub_score
pub_misscore

0.40408580183861087